# Import all necessary libraries 

In [1]:
from datascience import *
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import requests
import re
import numpy as np 
from numpy import argsort, searchsorted, array, size

import folium
import pandas as pd
from shapely.geometry import Point, Polygon

# Define a class 'tools' with convenient functions

In [41]:
class Tools:
    # Helper functions to trim whitespaces
    def trim_l(self, line):
        return re.sub(r'^\s+', '', line)

    def trim_r(self, line):
        return re.sub(r'\s+$', '', line)

    def trim(self, line):
        return self.trim_l(self.trim_r(line))

    # Check if the line is the start of a node
    def is_node_s(self, a):
        return re.match(r'^.*<node[\s+,>].*$', a, flags=re.IGNORECASE) is not None

    # Check if the line is the end of a node
    def is_node_e(self, a):
        return re.match(r'^.*</node>.*$', a, flags=re.IGNORECASE) is not None

    # Check if the line is the start of an element (way or relation)
    def is_elem_s(self, tag, a, flags=re.IGNORECASE):
        return re.match(r'^.*<' + str(tag) + '[\s+,>].*$', a, flags=flags) is not None

    # Check if the line is the end of an element (way or relation)
    def is_elem_e(self, tag, a, flags=re.IGNORECASE):
        return re.match(r'^.*</' + str(tag) + '>.*$', a, flags=flags) is not None

    # Extract node ID, latitude, and longitude from a line
    def getnode_id_lon_lat(self, line):
        xid = re.sub(r'^.*\s+id="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"\s+.*$', r'\1', line)
        xlon = re.sub(r'^.*\s+lon="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"\s+.*$', r'\1', line)
        xlat = re.sub(r'^.*\s+lat="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"\s+.*$', r'\1', line)
        return int(xid), float(xlat), float(xlon)

    # Extract ID from an element (node, way, relation)
    def getelem_id(self, line):
        xid = re.sub(r'^.*\s+id="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"\s+.*$', r'\1', line)
        return int(xid)

    # Extract tag data from a line
    def gettag_int(self, tag, line):
        xid = re.sub(r'^.*\s+' + str(tag) + '="([-+]?[0-9]*)"\s+.*$', r'\1', line)
        return int(xid)

    # Open and read the local file
    def get_osm(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            Text = file.read().splitlines()  # Read all lines and split by newline

        # 1. To extract "node" information (id, lat, lon) and data (id_ref, tags)
        node_ref = []
        node_lon = []
        node_lat = []
        node_ref1 = []
        node_info = []

        nl = 0
        flag_s = flag_e = False
        xid = xlat = xlon = 0
        xdata = ''
        flag_open = False

        for line in Text:
            nl += 1
            flag_s = self.is_node_s(line)
            flag_e = self.is_node_e(line)
            if flag_s:
                flag_open = True
                xid, xlat, xlon = self.getnode_id_lon_lat(line)
                node_ref.append(xid)
                node_lat.append(xlat)
                node_lon.append(xlon)
                xdata = ''
            elif flag_e:
                flag_open = False
                node_ref1.append(xid)
                node_info.append(xdata)
            else:
                if flag_open:
                    xdata += str(self.trim(line))

        print(f"{1:4d} {'Reading Nodes over No. lines':40} {nl:15d}")
        print(f"{'No. Nodes =':45s} {len(node_ref):15d}")

        # 2. To extract "way" information (id) and way data (node_ref, tags, etc)
        way_ref = []
        way_info = []

        nl = 0
        flag_s = flag_e = False
        wid = 0
        wdata = ''
        flag_open = False

        for line in Text:
            nl += 1
            flag_s = self.is_elem_s('way', line)
            flag_e = self.is_elem_e('way', line)
            if flag_s:
                flag_open = True
                wid = self.getelem_id(line)
                way_ref.append(wid)
                wdata = ''
            elif flag_e:
                flag_open = False
                way_info.append(wdata)
            else:
                if flag_open:
                    wdata += str(self.trim(line))

        print(f"{2:4d} {'Reading Way over No. lines':40} {nl:15d}")
        print(f"{'No. Way =':45s} {len(way_ref):15d}")

        # 3. To extract "relation" information (id) and data (ref, tags, etc)
        rel_ref = []
        rel_info = []

        nl = 0
        flag_s = flag_e = False
        rid = 0
        rdata = ''
        flag_open = False

        for line in Text:
            nl += 1
            flag_s = self.is_elem_s('relation', line)
            flag_e = self.is_elem_e('relation', line)
            if flag_s:
                flag_open = True
                rid = self.getelem_id(line)
                rel_ref.append(rid)
                rdata = ''
            elif flag_e:
                flag_open = False
                rel_info.append(rdata)
            else:
                if flag_open:
                    rdata += str(self.trim(line))

        print(f"{3:4d} {'Reading Relation over No. lines':40} {nl:15d}")
        print(f"{'No. Relation =':45s} {len(rel_ref):15d}")

        return np.array(node_ref, dtype='int64'), \
               np.array(node_ref1, dtype='int64'), \
               np.array(way_ref, dtype='int64'), \
               np.array(rel_ref, dtype='int64'), \
               np.array(node_lon, dtype='float64'), \
               np.array(node_lat, dtype='float64'), \
               node_info, \
               way_info, \
               rel_info
    

    def extract_districts(self, rel_info):
        quan_lst = []
        quan_ref = []

        for k in range( len(rel_info) ):
            line = rel_info[k]
            if ( re.match( r'^.*Quận.*$', line, flags=re.IGNORECASE ) is not None ) :
                if ( re.match( r'^.*<tag k="name" v="Quận\s+.*$', line, flags=re.IGNORECASE ) is not None ) :
                    quan = re.sub( r'^.*(<tag k="name" v=")(Quận\s+(\d+|[A-Za-zÀ-ỹ\s]+))("/>.*)$', r'\2', line )
                    if ( quan is not None ):
                        quan_lst.append ( quan )
                        quan_ref.append ( k )
                
        return quan_lst, quan_ref
    
    
    
    # searching the table by ID
    def table_search(self, t, xid, col=0):
        if col == 0:
            k = searchsorted(t.column(col), xid)
            return t.take(k)        
        else:
            ia = argsort(t.column(col))
            k = searchsorted(t.column(col), xid, sorter=ia)
            return t.take(ia[k])

    # Get index from ref 
    def find_id_from_ref(self, aref, iref, getsorted=True):
        if getsorted:
            k = searchsorted(aref, iref)
        else:
            ia = argsort(aref)
            k = searchsorted(aref, iref, sorter=ia)
        return k

    
    def get_values_from_ref(self, aval, aref, iref):
        if size(iref) == 1:        
            iid = self.find_id_from_ref(np.dtype('int64').type(aref), iref, getsorted=True)
            return array(aval)[iid]
        else:
            list_id = [] 
            for j in range(size(iref)):
                list_id.append(self.find_id_from_ref(np.dtype('int64').type(aref), iref[j], getsorted=True))
            return array(aval)[list_id]
        
    def get_rel_from_ix(self, ref, rel_info):
        return rel_info[ref]
    
    
    # way

    # extract tag with key-value pair
    def way_has_tags(self, wval):
        tag_dict = {}
        for k, v in re.findall(r'<tag k="([^"]+)" v="([^"]+)"', wval):
            if k not in tag_dict:
                tag_dict[k] = v
            else:
                tag_dict[k].append(v)  
        return tag_dict 
    
    # extract all node refs
    def way_has_nodes(self, wval):
        nodes = []
        for x in re.findall(r'<nd\b(.*?)\/>', wval):
            nodes.append(self.trim(x))
        return nodes

    # convert all node refs to int and store in an array 
    def way_has_ref_of_node(self, wval):
        def get_ref(ele):
            k = re.sub(r'^ref="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"$', r'\1', ele)
            return np.dtype('int64').type(k)
        
        rnodes = []
        for x in self.way_has_nodes(wval):
            rnodes.append(get_ref(self.trim(x)))
        return np.dtype('int64').type(rnodes)
    
    # return node refs and tags pair 
    def way_has_refs_n_tags(self, wval):
        tag_dict = self.way_has_tags(wval)  
        return self.way_has_ref_of_node(wval), tag_dict
    
    # find ix of way from ref 
    def wayid_from_ref(self, wref):
        return self.find_id_from_ref(np.dtype('int64').type(way_ref), wref, getsorted=True)

    # extract info from ref 
    def wayinfo_from_ref(self, wref):
        return way_info[self.wayid_from_ref(wref)]
    
    
    
    # Input is relation infor 
    # return node refs , way refs, relation refs, tag with key-value pair
    def get_has_refs_n_tags(self, rquan):
        node_ref_pattern = r'<member type="node" ref="(\d+)"'
        way_ref_pattern = r'<member type="way" ref="(\d+)"'
        relation_ref_pattern = r'<member type="relation" ref="(\d+)"'
        tag_pattern = r'<tag k="([^"]+)" v="([^"]+)"'

        node_refs = list(map(int, re.findall(node_ref_pattern, rquan)))
        way_refs = list(map(int, re.findall(way_ref_pattern, rquan)))
        relation_refs = list(map(int, re.findall(relation_ref_pattern, rquan)))
        tags = re.findall(tag_pattern, rquan)

        tag_dict = {k: v for k, v in tags}

        return node_refs, way_refs, relation_refs, tag_dict
    
    # get coordinates(lat,lon) for all node refs input
    def get_coordinates_for_nodes(self, wn, node_ref, node_lat, node_lon):
        line_coordinates = []
        for r in wn:
            xlat = self.get_values_from_ref(node_lat, node_ref, r)
            xlon = self.get_values_from_ref(node_lon, node_ref, r)
            line_coordinates.append([xlat, xlon])
        return line_coordinates

    # input is all way refs in relation 
    def get_lines(self, qw, node_ref, node_lat, node_lon):
        qw_lines = [] 
        for j in range(len(qw)):
            wref = qw[j]
            wval = self.wayinfo_from_ref(wref)
            wn, _ = self.way_has_refs_n_tags(wval)
            line_coordinates = self.get_coordinates_for_nodes(wn, node_ref, node_lat, node_lon)
            qw_lines.append(line_coordinates)

        return qw_lines
    
    # this function ensures that all coordiantes continuous and closed polygon  
    def close_polygon(self, boundary_lines):
        blines = np.copy(np.array(boundary_lines[0]).reshape((-1, 2)))
        btail = blines[-1]  

        flags = np.zeros(len(boundary_lines))  
        flags[0] = 1  

        while np.count_nonzero(flags) < len(boundary_lines):
            hmin = 1.0e+10
            tmin = 1.0e+10
            ih = it = -1

            for n in range(1, len(boundary_lines)):
                if flags[n] > 0:
                    continue  

                xlines = np.copy(np.array(boundary_lines[n]).reshape((-1, 2)))
                xhead = xlines[0]  
                xtail = xlines[-1]  

                h1 = np.max(np.abs(xhead - btail))
                t1 = np.max(np.abs(xtail - btail))

                if h1 < hmin:
                    ih = n
                    hmin = h1

                if t1 < tmin:
                    it = n
                    tmin = t1

            if hmin < tmin:
                n = ih
                xlines = np.copy(np.array(boundary_lines[n]).reshape((-1, 2)))
                blines = np.append(blines, xlines, axis=0)
                btail = blines[-1]
            else:
                n = it
                xlines = np.copy(np.array(boundary_lines[n]).reshape((-1, 2)))
                blines = np.append(blines, xlines[::-1], axis=0) 
                btail = blines[-1]

            flags[n] = 1

        closed_polygon = np.append(blines, blines[0]).reshape((-1, 2))

        return closed_polygon
    
    
    # extract info in way info following tag_key and tag_value pair
    # return a table includes way ref, node refs and name of this way columns
    def extract_way_info(self, way_table, tag_key, tag_value):
    
        def parse_way_info(way_info, tag_key, tag_value):
            tag_pattern = rf'<tag k="{tag_key}" v="{tag_value}"/>'
            if re.search(tag_pattern, way_info):
                node_refs = [int(ref) for ref in re.findall(r'<nd ref="(\d+)"/>', way_info)]
                name = re.search(r'<tag k="name" v="([^"]+)"/>', way_info)
                name_value = name.group(1) if name else None
                return node_refs, name_value
            return None, None

        way_refs = way_table.column('ref')
        way_infos = way_table.column('info')

        filtered_data = []

        for ref, info in zip(way_refs, way_infos):
            node_refs, name = parse_way_info(info, tag_key, tag_value)
            if node_refs is not None:
                filtered_data.append((ref, node_refs, name))

        return Table().with_columns(
            'ref', [row[0] for row in filtered_data],
            'node_refs', [row[1] for row in filtered_data],
            'name', [row[2] for row in filtered_data]
        )
    
    
    # extract center point of a polygon
    def get_polygon_center(self, polygon_coords):
        polygon = Polygon(polygon_coords)
        centroid = polygon.centroid
        
        return (centroid.x, centroid.y)

tools = Tools()  

----
# Read osm file to extract data

----

In [4]:
node_ref, node_ref1, way_ref, rel_ref,\
node_lon, node_lat, node_info, way_info, rel_info = tools.get_osm('data/data_district_7.osm')

   1 Reading Nodes over No. lines                      553471
No. Nodes =                                            155096
   2 Reading Way over No. lines                        553471
No. Way =                                               29779
   3 Reading Relation over No. lines                   553471
No. Relation =                                            633


----
# Combine data into tables form and save them in CSV files

----

### _t_node table (node_ref,  lat and lon )_

In [5]:
t_node = Table().with_columns(
    'ref', node_ref,
    'lat', node_lat,
    'lon', node_lon
).sort( 'ref' )

print ( t_node.num_rows  )

#save
df = t_node.to_df()
df.to_csv('data/t_node.csv', index=False)

t_node.show(3)

155096


ref,lat,lon
366367233,10.7711,106.71
366367242,10.7093,106.737
366367408,10.7495,106.723


### _t_node_info table (node_info_ref,  info )_

In [6]:
t_node_info = Table().with_columns(
    'ref', node_ref1,
    'info', node_info
).sort( 'ref' )

print ( t_node_info.num_rows  )

#save
df = t_node_info.to_df()
df.to_csv('data/t_node_info.csv', index=False)

t_node_info.show(3)

13374


ref,info
366370651,"<tag k=""highway"" v="" ..."
366370764,"<tag k=""highway"" v="" ..."
366374711,"<tag k=""highway"" v="" ..."


### _t_way table (way_ref,  info )_

In [7]:
t_way = Table().with_columns(
    'ref', way_ref,
    'info', way_info
).sort( 'ref' )

print ( t_way.num_rows  )

#save
df = t_way.to_df()
df.to_csv('data/t_way.csv', index=False)

t_way.show(3)

29779


ref,info
32575751,"<nd ref=""313 ..."
32575869,"<nd ref=""5738 ..."
32575895,"<nd ref=""366 ..."


### _t_rel table  (rel_ref,  info )_

In [8]:
t_rel = Table().with_columns(
    'ref', rel_ref,
    'info', rel_info
).sort( 'ref' )

print ( t_rel.num_rows  )

#save
df = t_rel.to_df()
df.to_csv('data/t_rel.csv', index=False)

t_rel.show(3)

633


ref,info
49915,<t ...
1435294,<member ...
1904421,<member ...


----
# Read CSV files

----

In [9]:
node_table = Table.read_table('data/t_node.csv')
node_table

ref,lat,lon
366367233,10.7711,106.71
366367242,10.7093,106.737
366367408,10.7495,106.723
366367420,10.7404,106.686
366367523,10.762,106.675
366367541,10.7409,106.687
366367658,10.7514,106.679
366367908,10.7536,106.68
366368084,10.7435,106.714
366368291,10.7788,106.765


In [10]:
node_info_table = Table.read_table('data/t_node_info.csv')
node_info_table

ref,info
366370651,"<tag k=""highway"" v="" ..."
366370764,"<tag k=""highway"" v="" ..."
366374711,"<tag k=""highway"" v="" ..."
366375698,"<tag k=""highway"" v="" ..."
366376465,"<tag k=""highway"" v="" ..."
366383255,"<tag k=""highway"" v="" ..."
366383451,"<tag k=""highway"" ..."
366386678,"<tag k=""highway"" v="" ..."
366387791,"<tag k=""highway"" v="" ..."
366388361,


In [11]:
way_table = Table.read_table('data/t_way.csv')
way_table

ref,info
32575751,"<nd ref=""313 ..."
32575869,"<nd ref=""5738 ..."
32575895,"<nd ref=""366 ..."
32575935,"<nd ref=""320 ..."
32575985,"<nd ref=""6754 ..."
32576047,"<nd ref=""881 ..."
32576407,"<nd ref=""3663 ..."
32576414,"<nd ref=""268 ..."
32576600,"<nd ref=""57628 ..."
32576644,"<tag k=""high ..."


In [12]:
relation_table = Table.read_table('data/t_rel.csv')
relation_table

ref,info
49915,<t ...
1435294,<member ...
1904421,<member ...
1973756,"<member type=""node"" ref=""369486734"" role=""admin_centre""/ ..."
2204742,<member ...
2210236,<member ...
2531662,"<tag k="" ..."
2559086,<mem ...
2561415,<mem ...
2561568,<mem ...


### _Find all districts into dataset_

In [13]:
dist_list, dist_ix = tools.extract_districts(relation_table[1])

In [14]:
districts_table = Table().with_columns(
    'index', dist_ix,
    'District Name', dist_list
).sort( 'index' )

districts_table

index,District Name
14,Quận 1
20,Quận 7
31,Quận 4
45,Quận 3
46,Quận 5
78,Quận 10
83,Quận 8


#### _My area is district 7 only_

In [15]:
district_7 = tools.table_search(districts_table, 20)
district_7

index,District Name
20,Quận 7


----
# Data execution

----

### Get relation of district 7

In [16]:
district_7_ix = district_7[0][0]

rel_district_7 = relation_table.take(district_7_ix)[1][0]

rel_district_7


'<member type="way" ref="205798561" role="outer"/><member type="way" ref="205958089" role="outer"/><member type="way" ref="205798559" role="outer"/><member type="way" ref="205798574" role="outer"/><member type="way" ref="532477270" role="outer"/><member type="way" ref="205798562" role="outer"/><member type="way" ref="289880833" role="outer"/><member type="way" ref="289941958" role="outer"/><member type="way" ref="205798563" role="outer"/><member type="way" ref="290064899" role="outer"/><member type="way" ref="192036001" role="outer"/><member type="way" ref="192036002" role="outer"/><member type="way" ref="205798560" role="outer"/><member type="way" ref="418085082" role="outer"/><member type="way" ref="205798578" role="outer"/><member type="way" ref="206677429" role="outer"/><member type="way" ref="205798579" role="outer"/><member type="way" ref="205798575" role="outer"/><member type="way" ref="206677437" role="outer"/><member type="way" ref="502262238" role="outer"/><member type="way" 

## arrange data into relation of district 7

In [17]:
dist_n, dist_w, dist_r, dist_t = tools.get_has_refs_n_tags(rel_district_7)

## Find and plot district 7's center point

#### find lat and lon of center point district 7

In [39]:
dist_n[0]

11214914777

In [18]:
dist_lat = tools.get_values_from_ref( node_lat, node_ref, dist_n[0] ) 
dist_lon = tools.get_values_from_ref( node_lon, node_ref, dist_n[0] )  

map_center = [ dist_lat, dist_lon ]

print ( 'center point = ( %15.10f,  %15.10f )\t node reference = %d ' % ( dist_lat, dist_lon, dist_n[0] ) ) 

center point = (   10.7375481000,   106.7302238000 )	 node reference = 11214914777 


#### plot this point on map

In [19]:
district = folium.Map ( location=map_center, zoom_start=13)

folium.Marker(
    location=map_center,
    tooltip="Click me!",
    popup="Center of district 7",
    icon=folium.Icon(color="purple"),
).add_to(district)

# district.save ( "data/images/center_of_district_7.html" )

district 

## get district 7's boundary  and plot

#### _get boundary_

In [20]:
boundary_dist_7 = tools.close_polygon( tools.get_lines( dist_w, node_ref, node_lat, node_lon ) )

#### _plot on map with polygon line_

In [21]:
folium.Polygon(
    locations = boundary_dist_7,
    color = "blue", 
    weight=2        
).add_to(district)

# district.save ( "data/images/boundary_of_district_7.html" )


district  

## Find all wards in district 7

In [22]:
ward_refs_in_dist_7 = dist_r
ward_ixs_in_dist_7 = [tools.find_id_from_ref(rel_ref, ref) for ref in ward_refs_in_dist_7]

ward_ns = []
ward_ws = []
ward_ts = []
ward_bounds = []

for ix in ward_ixs_in_dist_7:
    ward_rel = tools.get_rel_from_ix(ix, rel_info)
    ward_n, ward_w, _, ward_t = tools.get_has_refs_n_tags(ward_rel)
    ward_bound = tools.get_lines( ward_w, node_ref, node_lat, node_lon )
    ward_ns.append(ward_n[0])
    ward_ws.append(ward_w)
    ward_ts.append(ward_t['name'])
    ward_bounds.append(ward_bound)
    

wards_table = Table().with_columns(
    'index', ward_ixs_in_dist_7,
    'ward_ref', ward_refs_in_dist_7,
    'node_ref', ward_ns,
    'way_refs', ward_ws,
    'name', ward_ts,
    'boundary', ward_bounds
).sort('index')

In [23]:
wards_table

index,ward_ref,node_ref,way_refs,name,boundary
21,2766947,10166760587,"[205958109, 205958091, 205958108, 205958106, 205958107]",Phường Bình Thuận,"[[[10.7383735, 106.7159346], [10.7380265, 106.7216411999 ..."
22,2766948,10166760588,"[205958090, 205798559, 205798574, 532477270, 289854278]",Phường Phú Mỹ,"[[[10.7127242, 106.7512311], [10.7134383, 106.7431980000 ..."
23,2766949,10166760596,"[205958090, 205958112, 205958088, 205798561, 205958089]",Phường Phú Thuận,"[[[10.7127242, 106.7512311], [10.7134383, 106.7431980000 ..."
24,2766950,10166760591,"[205958098, 205958123, 289941957, 205798563, 290064899, ...",Phường Tân Hưng,"[[[10.7525084, 106.7050033], [10.7520931, 106.70499], [1 ..."
25,2766951,10166760592,"[418085082, 205958098, 205958100, 205958108, 205958087, ...",Phường Tân Kiểng,"[[[10.7525084, 106.7050033], [10.7524958, 106.7058182], ..."
26,2766952,10166760590,"[289880833, 205958105, 205958097, 289941957, 289941958]",Phường Tân Phong,"[[[10.7223883, 106.7203751], [10.7222109, 106.7198944], ..."
27,2766953,10166760589,"[289854278, 205798562, 205958105, 260850606, 205958109, ...",Phường Tân Phú,"[[[10.7176198, 106.7367933], [10.7177586, 106.7361504], ..."
28,2766954,10166760593,"[205958123, 205958100, 205958091, 260850606, 205958097]",Phường Tân Quy,"[[[10.7449938, 106.7043422], [10.739906, 106.7035552]], ..."
29,2766955,10166760594,"[205958110, 205798579, 205958087, 205958106]",Phường Tân Thuận Tây,"[[[10.7495081, 106.72873920000001], [10.7533463, 106.728 ..."
30,2766956,10166760595,"[205958088, 205958107, 205958110, 205798575, 206677437, ...",Phường Tân Thuận Đông,"[[[10.7375361, 106.7304317], [10.7378178, 106.7308426], ..."


## Plot all wards on map with polygon lines

In [24]:
for i in range(wards_table.num_rows): 
    
    row_i = wards_table.take(i)
    
    ward_lat = tools.get_values_from_ref(node_lat, node_ref, row_i['node_ref'])
    ward_lon = tools.get_values_from_ref(node_lon, node_ref, row_i['node_ref'])

    ward_center = [ward_lat, ward_lon]


    folium.Marker(
        location=ward_center,
        tooltip="Click me!",
        popup= row_i['name'][0],  
        icon=folium.Icon( color="cadetblue"),
    ).add_to(district)
        
    ward_boundary = tools.close_polygon(row_i['boundary'][0])
    folium.Polygon(
        locations = ward_boundary,
        color = "blue", 
        weight=2        
    ).add_to(district)
    

# district.save ( "data/images/boundary_of_wards`(no_color)`.html" )
    

district

## Plot the wards' boundaries with a filled color

In [25]:
colors = [
    'lightgreen',   
    'red',  
    'green',       
    'pink',        
    'cadetblue',   
    'lightblue',   
    'blue',       
    'darkred',     
    'orange',      
    'beige'        
]

for i in range(wards_table.num_rows):
    row_i = wards_table.take(i)

    ward_lat = tools.get_values_from_ref(node_lat, node_ref, row_i['node_ref'])
    ward_lon = tools.get_values_from_ref(node_lon, node_ref, row_i['node_ref'])
    ward_center = [ward_lat, ward_lon]


    folium.Marker(
        location=ward_center,
        tooltip="Click me!",
        popup=row_i['name'][0],  
        icon=folium.Icon(icon="cloud", color = colors[i]),
    ).add_to(district)

    ward_boundary = wards_table['boundary'][i]

    folium.Polygon(
        locations=tools.close_polygon(ward_boundary),  
        color="blue", 
        weight=2,         
        fill=True,        
        fill_color = colors[i],  
        fill_opacity=0.9  
    ).add_to(district)
    
    
# district.save ( "data/images/boundary_of_wards`(with_color)`.html" )

district


## Create district7's boundary polygon

In [26]:
boundary_district_7_polygon = Polygon(boundary_dist_7)

# Extract all shcools and plot on map

### _Create university table_

In [27]:
unis_table = tools.extract_way_info(way_table, "amenity", "university")
unis_table.show(3)

ref,node_refs,name
140703173,"[1541029340, 9947678197, 9947678199, 9947678198, 9947678 ...",Trường Đại học Sư phạm TP.HCM
140703186,"[1541029452, 9947678127, 9947678126, 1541029494, 1541029 ...",Trường Đại học Khoa học Tự nhiên - ĐHQG TP.HCM - Cơ sở 1
149904631,"[1628403135, 1628403136, 1628403134, 1628403133, 1628403 ...",CĐ nghề GTVT đường thủy II


### _Create a table that includes all schools except universities_


In [28]:
schools_table = tools.extract_way_info(way_table, "amenity", "school")
schools_table.show(3)

ref,node_refs,name
140703181,"[1541029611, 1541029607, 4623130693, 1541029604, 1541029 ...",Trường THPT chuyên Lê Hồng Phong
140703185,"[1541029266, 11989673568, 11989673567, 8642545378, 15410 ...",Trường Trung học Thực hành Sài Gòn
149904632,"[1628403385, 1628403393, 1628403372, 1628403362, 1628403385]",Trường Hàn Quốc HCMC Korean School


#### _combine 2 table into an array_

In [29]:
edu_table = [unis_table, schools_table]

## Plot all of them on map

In [30]:
edu_map = folium.Map(location=map_center, zoom_start=13 )

colors = [
    'darkred',   
    'cadetblue'        
]
 
# darkred color is universities
# cadetblue color is others school


for i, table in enumerate(edu_table):
    color = colors[i]
    for i in range( table.num_rows ):
        boundary = tools.close_polygon(tools.get_lines( [table.row(i)[0]], node_ref, node_lat, node_lon ))
        if np.shape(boundary)[0] >= 4:
            name = table.row(i)[2]
            if name:
                boundary_center = tools.get_polygon_center(boundary)
                center_point = [boundary_center[0], boundary_center[1]]  
                if boundary_district_7_polygon.contains(Point(center_point)):
                    folium.Marker(
                        location=center_point,  
                        tooltip="Click me!",
                        popup= name,  
                        icon=folium.Icon(icon="cloud", color=color),
                    ).add_to(edu_map)

                    folium.Polygon(
                        locations=boundary,  
                        color="blue", 
                        weight=2,         
                        fill=True,        
                        fill_color = color,  
                        fill_opacity=0.7  
                    ).add_to(edu_map)

folium.Polygon(
    locations = boundary_dist_7,
    color = "blue", 
    weight=2        
).add_to(edu_map)

# edu_map.save ( "data/images/education.html" )


edu_map

## Extract some of places 

### _hospital table_

In [31]:
hel_table = tools.extract_way_info(way_table, "amenity", "hospital")
hel_table.show(3)

ref,node_refs,name
158708608,"[1708640387, 2322640528, 1708640373, 10761763039, 170864 ...",Bệnh Viện Pháp Việt
158971987,"[1710814099, 3663992202, 1710814096, 10777654810, 170864 ...",Bệnh Viện Tim Tâm Đức
187949834,"[1985832561, 7503285427, 7503285426, 1985832668, 1985832 ...",Bệnh viện Đa khoa Sài Gòn


### _park table_

In [32]:
park_table = tools.extract_way_info(way_table, "leisure", "park")
park_table.show(3)

ref,node_refs,name
32735084,"[2956813610, 2956817445, 368339701]",Khu du lịch Kỳ Hòa
149904638,"[1628403155, 1628403154, 1628403144, 1628403145, 1628403155]",Trung Tâm Văn Hóa Quận 7
165271483,"[1768268752, 1768268750, 1768268751, 1768268742, 1768268 ...",None


### _golf table_

In [33]:
golf_table = tools.extract_way_info(way_table, "leisure", "golf_course")
golf_table.show(3)

ref,node_refs,name
158710364,"[1708656421, 5372370418, 367321853, 367321856, 367321862 ...",Golf Nam Sai Gon
210778858,"[2208325312, 10776290609, 10776290608, 1756318867, 22083 ...",Golf Nam Sai Gon
218448425,"[2276969190, 2276969189, 2276969185, 2276969177, 2276969 ...",Sân Tập Golf


### _supermarket table_

In [34]:
supermarket_table = tools.extract_way_info(way_table, "shop", "supermarket")
supermarket_table.show(3)

ref,node_refs,name
152994798,"[1657536332, 1657536313, 1657536318, 1657536311, 1657536 ...",Co.op Mart Cống Quỳnh
290188110,"[2937460934, 2937460935, 2937460936, 2937460937, 2937460 ...",Citimart
509015321,"[4983065216, 4983065217, 4983065218, 4983065219, 4983065 ...",Vinmart (Maximark) 3 tháng 2


### _police table_

In [35]:
police_table = tools.extract_way_info(way_table, "amenity", "police")
police_table.show(3)

ref,node_refs,name
149904630,"[1628403149, 1628403146, 1628403141, 1628403142, 1628403 ...",Công An Quận 7
187895192,"[1985346271, 1985346092, 1985346153, 4611410090, 1985346 ...",Sở Công An TP.HCM
224449760,"[2332723074, 2332723071, 2332723070, 2332723072, 2332723074]",None


### _marketplace table_


In [36]:
marketplace_table = tools.extract_way_info(way_table, "amenity", "marketplace")
marketplace_table.show(3)

ref,node_refs,name
39514795,"[473382796, 2893838361, 2893838357, 2893838358, 23535415 ...",Chợ Bến Thành
79345731,"[926558653, 1777244267, 1777244405, 926558632, 177724438 ...",Chợ Xóm Chiếu
152992729,"[1657501021, 1657501019, 1657501017, 1657501025, 4613361 ...",Chợ hoa Hồ Thị Kỷ


#### _combine all tables into an array_

In [37]:
others_table = [hel_table, park_table, golf_table, supermarket_table, police_table, marketplace_table]

### Plot all of them on map

In [38]:
others_map = folium.Map(location=map_center, zoom_start=13 )

colors = [
    'lightgreen',   
    'red',  
    'green',       
    'pink',        
    'blue',       
    'beige'        
]
 
# lightgreen color is hospitals
# red color is parks
# green color is golfs 
# pink color  is supermaker 
# blue color is police stations
# beige color is marketplace 



for i, table in enumerate(others_table):
    color = colors[i]
    for i in range( table.num_rows ):
        boundary = tools.close_polygon(tools.get_lines( [table.row(i)[0]], node_ref, node_lat, node_lon ))
        if np.shape(boundary)[0] >= 4:
            name = table.row(i)[2]
            if name:
                boundary_center = tools.get_polygon_center(boundary)
                center_point = [boundary_center[0], boundary_center[1]]  
                if boundary_district_7_polygon.contains(Point(center_point)):
                    folium.Marker(
                        location=center_point,  
                        tooltip="Click me!",
                        popup= name,  
                        icon=folium.Icon(color=color),
                    ).add_to(others_map)

                    folium.Polygon(
                        locations=boundary,  
                        color="blue", 
                        weight=2,         
                        fill=True,        
                        fill_color = color,  
                        fill_opacity=0.7  
                    ).add_to(others_map)

folium.Polygon(
    locations = boundary_dist_7,
    color = "blue", 
    weight=2        
).add_to(others_map)

# others_map.save ( "data/images/amenities.html" )


others_map

![benign](data)

In [44]:
!ls data/images/amenities.html 

amenities.html                      boundary_of_wards (with_color).html
boundary_of_district_7.html         center_of_district_7.html
boundary_of_wards (no_color).html   education.html
